# Setup

### Run Previous Scripts

In [1]:
%run ./1_Master_Script.ipynb
%run ./2_Cleaning_Layers.ipynb
%run ./3.1_City_Scraping.ipynb
%run ./3.2_City_Cleaning.ipynb

Master Script has been run successfully!
Cleaning Layers Script has been run successfully!
Chosen Study Area: Stolen Lands
Days since last generation: 0
Regenerate Cities: False
City Scraping Script has been run successfully!
RESULT: From 851 cities, 835 were removed, leaving just 16.
RESULT: From 16 cities, 2 were removed, leaving just 14.
City Cleaning Script has been run successfully!


#### Raster Layers

What supply costs should each layer have? That's the purpose of this dictionary.

The layers are divided into separate dictionaries, which will then be combined in a list. This is for organizational purposes, as I plan to add many more layers.

The point of comparison is a cost of 1. It costs one unit of supplies to traverse ordinary plains. Each additional layer is additive, increasing the cost by a fixed amount.

In the future, more terrain layers may be added. For example, making thin slivers of coastline (aside from coastlines adjacent to cities) difficult to traverse, but the sea past that much easier than land to traverse, so that sea routes are encouraged across long distances.

In [2]:
terrain_layers = {
   'deserts': {
       'data': final_dict['deserts'],
       'cost': 1,
       'color': 'brown'
    },
    'forests': {
        'data': final_dict['forests'],
        'cost': 1,
        'color': 'green'
    },
    'hills': {
        'data': final_dict['hills'],
        'cost': 1,
        'color': 'darkgreen'
    },
    'ice': { # These are glaciers
        'data': final_dict['ice'],
        'cost': 1,
        'color': 'white'
    },
    'borders': { # Borders, ironically, lines up with land better than land!
        'data': final_dict['borders'],
        'cost': 1,
        'color': 'yellow'
    },
    'mountains': {
        'data': final_dict['mountains'],
        'cost': 1,
        'color': 'red'
    },
    'rivers': {
        'data': final_dict['rivers'],
        'cost': 5, # To make it not easy to cross, requiring a bridge
        'color': 'blue'
    },
    'swamps': {
        'data': final_dict['swamps'],
        'cost': 1,
        'color': 'darkblue'
    },
    'waters': {
        'data': final_dict['waters'],
        # IDEA: create coastline geometry and assign that a high weight, but then make all other waters low weight.
        # This is to account for the startup cost of sailing, and how once you've departed you can go a long distance
        'cost': 10,
        'color': 'lightblue'
    }
}

### Cleaned Cities

In [3]:
final_cities.head(2)

,Name,link,capital,size,text,articleLength,geometry,Nation,Population,Government,...,Region,Religions,Ruler,Languages,Capital,Titles,Demonym,Demographics,Adjective,size0
0,Avendale,https://pathfinderwiki.com/wiki/Avendale,False,1,<p><b>Avendale</b> is the capital city of <a h...,600,POINT (-529191.389 1902575.629),River Kingdoms,11280,Military dictatorship,...,Touvette,<NA>,General Cabol Voran,<NA>,<NA>,<NA>,<NA>,"Human 89%, halfling 9%, other 2%",<NA>,False
1,Brunderton,https://pathfinderwiki.com/wiki/Brunderton,False,2,<p>The mining town of <b>Brunderton</b> in eas...,300,POINT (-4315.401 2126693.082),Brevoy,1120,<NA>,...,Rostland,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Overwhelmingly dwarf,<NA>,False


### Interactive Map

This is our equivalent to Google Maps, for finding point coordinates.

https://map.pathfinderwiki.com/#location=4.69/47.4/7.93

# Cost Distance Map

#### Remove Empty Layers

Some layers may have no data by this point. Let's remove them now, for future ease.

In [4]:
def remove_empty_layers(dictionary):

    new_dict = dictionary.copy()
    
    for item in dictionary:

        if dictionary[item]['data'].empty == True:
            new_dict.pop(item) # Gets rid of that item!
        else:
            continue

    return new_dict

In [5]:
terrain_layers = remove_empty_layers(terrain_layers)

terrain_layers.keys()

dict_keys(['forests', 'hills', 'borders', 'mountains', 'rivers', 'swamps', 'waters'])

#### Initialize Google Earth Engine

NOTE: before launching Jupyter Lab, you MUST enter "earthengine authenticate" in the Conda terminal.

In [6]:
ee.Authenticate() # Substitutes for entering it in the console manually, I believe

# Initialize the Earth Engine module.
ee.Initialize()

### Functions

#### GPD to EE

In [7]:
def gpd_to_ee(data):

    ee_object = geemap.geopandas_to_ee(gdf = data)
    
    return ee_object

#### Add Property

This function takes a feature, and assigns it a 'cost' of a certain value. The intent is to loop this across a feature collection via .map().

In [8]:
# Why two functions? .map() does not really accept arguments other than data source, so defining the loop function within this function
# allows me to have a function where cost is supplied as an argument, without having to do something cheap like calling cost as a
# global variable.
def add_property(data, property_name, property_value):

    def loop_function(data): # function to be looped across features

        result = data.set(property_name, property_value)

        return result

    result = data.map(loop_function)

    return result

#### Rasterize Feature

This function takes an ee.feature, and turns it into a raster. This has two main uses:

1) Constructing the blank raster. The blank raster is the raster upon which all additional rasters are added to, creating the cost distance map. It is created by rasterizing the bounding box.

2) Constructing individual layers. These are the layers being added to the blank raster.

In [9]:
# NOTE: might not work with feature collections! Make another function / include as argument if this is the case!
def rasterize_feature(data):

    result = data.reduceToImage(
        properties = ['cost'],
        # The reducer answers the question, 'what if there's an overlap?' The 'first' reducer simply favors based on order.
        reducer = ee.Reducer.first()
    )

    result = result.toInt() # CRITICAL: assumes that I'm using only integers, which so far is the plan!!

    return result

### Cleaning

This script calculates and plots the cost distance map and features. By default, the plotting part should be disabled. Why? Because, while the calculations are server-side, meaning they're quick, the plotting is necessarily client-side, so it's a *lot* of data your computer must now plot. My computer struggles to do that.

#### Create Study Area

This'll be useful throughout. Let's create and use the bouding box from before as a separate image.

In [10]:
study_ee = studyarea.copy()

study_ee = gpd_to_ee(study_ee)

#study_ee

In [11]:
run = False

if run == True:

    Map = geemap.Map()
    
    Map.addLayer(
        ee_object = ee.Image().paint(
            study_ee,
            color = 0, # Black
            width = 10
        ),
        #vis_params = cost_map_spec,
        name = 'Study Area',
        shown = True
    )
    
    Map.centerObject(
        ee_object = study_ee,
        zoom = 6
    )
    
    Map

#### Construct Cost Map

In [12]:
image_list = ee.List([])

feature_list = ee.List([]) # used for diagnostic check

layer_list = ee.List([]) # Used for iterating across layer names

layer_num = 0 # Assigning each feature a number, for coloring purposes

for item in terrain_layers:

    layer_num += 1

    layer_list = layer_list.add(item)

    # Transform layer from GDF to ee.Feature
    layer = terrain_layers[item]['data'].copy()
    layer = gpd_to_ee(layer)

    # Add properties
    layer = add_property(
        layer,
        property_name = 'cost',
        property_value = terrain_layers[item]['cost']
    )
    layer = add_property(
        layer,
        property_name = 'layer',
        property_value = item
    )
    layer = add_property(
        layer,
        property_name = 'layer_num',
        property_value = layer_num
    )
    layer = add_property(
        layer,
        property_name = 'color',
        property_value = terrain_layers[item]['color']
    )
    layer = layer.set({'layer': item}) # Applied at collection level, for better organization

    # Save the layer while it's still a feature, for diagnostics
    feature_list = feature_list.add(layer)
    TEST_DIAG = layer

    # Rasterize
    layer = rasterize_feature(layer)

    image_list = image_list.add(ee.FeatureCollection(layer))

input_images = ee.ImageCollection(image_list)
# DONT make features into a featurecollection, I want them as separate collections!
input_features = ee.FeatureCollection(feature_list).flatten()

cost_map = input_images.reduce(
    reducer = ee.Reducer.sum()
) # Maybe adding .reduce will make it server-side?

cost_map = cost_map.toInt() # Otherwise it cant export. Plus, for some reason, it defaults to long

#### Summary Stats (Disabled)

For diagnostics as well as the map, let's calculate summary statistics for the cost map.

### Diagnostic Map (Disabled)

This map plots each input feature, so it can be confirmed that the costs really make sense.

### Cost Map (Disabled)

# Export (Disabled)

Disabled because, with how the code works now, there's no need to export this to Google Drive. Though, the truth of the matter will be settled once the road network script gets developed. For that script, it may actually be better to export an image before calling it back, as, that way, the image is *forced* to take on the designated resolution.

# Run Message

This is to show key info when this script is run in another script.

In [13]:
print("Terrain Cost Distance Script has been run successfully!")

Terrain Cost Distance Script has been run successfully!
